## Snippet: Limits Profiles, Mixed Severity, and Frequency Mixing

The `Aggregate` distribution class manages creation and calculation of aggregate distributions.
It allows for very flexible creation of aggregate distributions, including  a limit profile, 
a mixed severity, or both. Mixed frequency types share
a mixing distribution across all broadcast terms to ensure an appropriate inter-
class correlation.

In [ ]:
sys.path.append('c:\\s\\telos\\python\\aggregate_project')
from aggregate import build
build.logger_level(30)

### Limit profiles
The exposure variables can be vectors to express a *limit profile*.
All ```exp_[en|prem|loss|count]``` related elements are broadcast against one-another.
For example

    agg Eg1 [100 200 400 100] premium at 0.65 lr [1000 2000 5000 4000] xs [0 0 0 1000] sev lognorm 500 cv 1.25 poisson

expresses a limit profile with 100 of premium at 1000 x 1000; 200 at 2000 x 1000
400 at 5000 x 1000 and 100 at 10000 x 1000. In this case all the loss ratios are
the same, but they could vary too, as could the attachments.

In [ ]:
# limit profile
eg1 = build('agg Eg1 [100 200 400 100] premium at 0.65 lr '
            '[1000 2000 5000 4000] xs [0 0 0 1000] '
            'sev lognorm 500 cv 1.25 '
            'poisson')
eg1.report_df

### Mixed severity distributions
The severity variables can be vectors to express a *mixed severity*. All severity
elements are broadcast against one-another. For example

    sev lognorm 1000 cv [0.75 1.0 1.25 1.5 2] wts [0.4, 0.2, 0.1, 0.1, 0.1]

expresses a mixture of five lognormals with a mean of 1000 and CVs as indicated with
weights 0.4, 0.2, 0.1, 0.1, 0.1. Equal weights can be expressed using the shorthand `wts=5`. 
A missing weights clause is interpreted as giving each severity weight 1. It results in five
times the total loss.

In [ ]:
# mixed severity
eg2 = build('agg Eg2 1000 loss sev lognorm 100 cv [0.75 1.0 1.25 1.5 2] wts [0.4, 0.2, 0.1, 0.1, 0.1] poisson')
eg2.report_df

In [ ]:
# mixed severity with poisson frequency is the same as the sum of five independent components
egPort = build('''
port EgPort
    agg Unit1 400 loss sev lognorm 100 cv 0.75 poisson
    agg Unit2 200 loss sev lognorm 100 cv 1.0 poisson
    agg Unit3 100 loss sev lognorm 100 cv 1.25 poisson
    agg Unit4 100 loss sev lognorm 100 cv 1.5 poisson
    agg Unit5 100 loss sev lognorm 100 cv 2.0 poisson

''')
egPort.report_df

In [ ]:
# actual frequency = total frequency x weight; wts=5 sets equal weights, here 0.2
eg2e = build('agg Eg2e 1000 loss sev lognorm 100 cv [0.75 1.0 1.25 1.5 2] wts=5 poisson')
eg2e.report_df

In [ ]:
# missing weights set to 1 resulting in five times loss
eg2m = build('agg Eg2m 1000 loss sev lognorm 100 cv [0.75 1.0 1.25 1.5 2] poisson')
eg2m.report_df

### Limit profiles and mixed severity
Limit profiles and mixtures can be combined. Each mixed severity is applied to each
limit profile component. For example

    agg multiExp [10 20 30] claims [100 200 75] xs [0 50 75] sev lognorm 100 cv [1 2] wts [.6 .4] mixed gamma 0.4

creates an aggregate with six severity subcomponents:

|  #  |   limit |   attachment |   claims |
|:---:|--------:|-------------:|---------:|
|  0  |     100 |            0 |        6 |
|  1  |     100 |            0 |        4 |
|  2  |     200 |           50 |       12 |
|  3  |     200 |           50 |        8 |
|  4  |      75 |           75 |       18 |
|  5  |      75 |           75 |       12 |

In [ ]:
# limits profile and mixed severity
eg3 = build('agg Eg3 [10 20 30] claims [100 200 75] xs [0 50 75] '
            'sev lognorm 100 cv [1 2] wts [0.6 0.4] '
            'poisson')
eg3.report_df

### Limit profiles with different severities: circumventing products
Exposures with different limits may have different severity curves. In this case, the limit profile 
and severity curves should all be broadcast together, rather than broadcasting limits and severities 
separately and then taking the outer product as in the previous example. 

    agg Eg4 [10 10 10] claims [1000 2000 5000] xs 0 \
        sev lognorm [50 100 150] cv [0.1 0.15 0.2] \
        poisson

The interpretation is determined by the total weights. If the weights sum to
one then the severity is interpreted as a mixture, and the result is an exposure / severity product as above. 
If the weights are missing (i.e., all equal 1 by default)  then the result is a different severity for each limit band. 
(If the weights are specified and sum to the number of severity components then they are used to adjust the 
expected losses. Usually, this is not the desired behavior.)

In [ ]:
# limits profile where each limit band has a different severity curve
eg4 = build('agg Eg4 [10 10 10] claims [1000 2000 5000] xs 0 '
            'sev lognorm [50 100 150] cv [0.1 0.15 0.2] '
            'poisson')
eg4.report_df

In [ ]:
# adding weights that sum to the number of components adjusted expected losses
eg4m = build('agg Eg4m [10 10 10] claims [1000 2000 5000] xs 0 '
            'sev lognorm [50 100 150] cv [0.1 0.15 0.2] wts [2 .5 .5]'
            'poisson')
eg4m.report_df

### Frequency mixing
All severity components in an aggregate share the same frequency mixing value, inducing correlation between
the parts. For example, to model the current accident year and prior year reserves. 

    agg Egn [100 200] claims sev [gamma lognorm] [100 150] cv [1 0.5] mixed gamma 0.4

`Egn` models the current accident year is gamma mean 100 cv 1 and a run-off reserve lognormal mean 150 cv 0.5.

In [ ]:
# mixed frequency, negative binomial cv 0.4
eg4x = build('agg Eg4x [1000 500 200 100] premium at [0.85 .75 .65 .55] lr '
            '[1000 2000 5000 10000] xs 1000 '
            'sev lognorm 100 cv .75 '
            'mixed gamma 0.4')
eg4x.report_df

In [ ]:
# model of current AY (gamma) and reserves(lognormal) with shared gamma mixing 
eg5 = build('agg Eg5 [100 200] claims 5000 x 0 sev [gamma lognorm] [100 150] cv [1 0.5] mixed gamma 0.5',
           log2=16, bs=2.5)
eg5.report_df

In [ ]:
# Delaporte (shifted) gamma mixing often produces more realistic output, avoiding very good years
eg5d = build('agg Eg5d [100 200] claims 5000 x 0 sev [gamma lognorm] [100 150] cv [1 0.5] mixed delaporte 0.5 0.6', 
            log2=18, bs=2.5)
eg5d.report_df

In [ ]:
eg5.plot()

In [ ]:
eg5d.plot()

Created: 2022-07-06 from `Aggregate` class docstring.